# Single Compartment Neural Simulator


## Importing relavant python libraries


In [1]:
import pandas as pd

try:
    from ipywidgets import widgets, Layout, interact, interactive, interactive_output, Dropdown
    from IPython.display import display
    import bqplot as bqp
    from bqplot import pyplot as plt # bqplot is a plotting for jupyter
    import matplotlib.pyplot as mplt
    import common
    import constants
    import compartment
    import electrodiffusion
    import numpy as np
    import random
    import seaborn as sns 
    import pandas as pd
    import graphing as gr
    #!pip install viola
    ########## Modules required for artist drawing on matplotlib
    import matplotlib.path as mpath
    import matplotlib.lines as mlines
    import matplotlib.patches as mpatches
    from matplotlib.collections import PatchCollection
    
  
    
    
except ModuleNotFoundError:
        print("A class you are trying to import is not present in the directory")
    
except Exception:
        print("Something went wrong - modules have not been imported")

else: 
        print("All relevant classes imported")
    


All relevant classes imported


## Setting Compartment parameters


In [31]:
### WIDGET INITIALIZATION ################################################################################

lbl_rad = widgets.Label(value = "Compartment Radius (um): ")
sldr_rad = widgets.FloatSlider(value=5, min=1, max=11.0, step=0.5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
rad_box = widgets.HBox(children = [lbl_rad,sldr_rad])

lbl_len = widgets.Label(value = "Compartment Length (um): ")
sldr_len = widgets.FloatSlider(value=100, min=50, max=200, step=20, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
len_box = widgets.HBox(children = [lbl_len,sldr_len])

lbl_nai = widgets.Label(value = "Na+ concentration (mM): ")
sldr_nai = widgets.FloatSlider(value=14, min=0, max=100, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_nai = widgets.HBox(children = [lbl_nai,sldr_nai])

lbl_ki = widgets.Label(value = "K+ concentration (mM): ")
sldr_ki = widgets.FloatSlider(value=122.9, min=0, max=250, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_ki = widgets.HBox(children = [lbl_ki,sldr_ki])

lbl_cli = widgets.Label(value = "Cl- concentration (mM): ")
sldr_cli = widgets.FloatSlider(value=5.2, min=0, max=50, step=5, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_cli = widgets.HBox(children = [lbl_cli,sldr_cli])

lbl_xi = widgets.Label(value = "Impermeant anion concentration (mM): ")
sldr_xi = widgets.FloatSlider(value=154.9, min=0, max=300, step=10, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.1f')
hbox_xi = widgets.HBox(children = [lbl_xi,sldr_xi])

lbl_zi = widgets.Label(value = "Impermeant anion average charge: ")
sldr_zi = widgets.FloatSlider(value=-0.85, min=-3, max=0, step=0.05, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_zi = widgets.HBox(children = [lbl_zi,sldr_zi])


lbl_p = widgets.Label(value = "ATPase pump constant (C/dm^2*s): ")
sldr_p = widgets.FloatSlider(value=0.1, min=0.05, max=1, step=0.05, disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='.2f')
hbox_p = widgets.HBox(children = [lbl_p,sldr_p])


btn_new_comp = widgets.Button(description = "Create Compartment",button_style ="success")
btn_defaults = widgets.Button(description = "Reset to defaults")
hbox_btns = widgets.HBox(children = [btn_new_comp, btn_defaults])

output1 = widgets.Output()

########################################################################################################




global R,F,T,RTF,g_x,g_na,g_k,g_cl
R = 8.31446 # gas constant
F = 96485.33  # Faraday's constant        C mol-1
T = 37 + 273.15 #Temperature
RTF = R * T / F
gx =0
gna = 2e-3/F #ionic conductances for each ion
gk =  7e-3/F
gcl = 2e-3/F 



df_sim = pd.DataFrame()


def new_comp_clicked(b):
    
    
    global df_sim
    
    radius = sldr_rad.value*1e-5
    length = sldr_len.value*1e-5
    w = np.pi * (radius ** 2) * length
    sa = 2 * np.pi * radius * length
    ar = sa/w #area scaling constant
    cm = 2e-4 #membrane capacitance 
    finvcar = F / (cm*ar)
    pkcc2 = 2e-3 / F
    p = sldr_p.value / F ### LOOK AT KIRAS SINGLE COMP
    
    nai = sldr_nai.value*1e-3
    ki = sldr_ki.value*1e-3
    cli = sldr_cli.value*1e-3
    xi = sldr_xi.value*1e-3
    zi = sldr_zi.value
    
    df_sim = [radius, length, w, sa, ar, finvar, pkcc2, p, nai, ki,cli,xi,zi ]
    df_sim.index = ['Radius', 'Length', 'Volume', 'Surface area', 'Finvar','pkcc2','p',Na_i', 'K_i', 'Cl_i', 'X_i', 'z_i', 'ATP pump rate', 'KCC2 pump rate',
                'Vm', 'Ek', 'ECl']
    
    
    with output1: 
        print("Compartment created")
    
    
    
      
btn_new_comp.on_click(new_comp_clicked)

def btn_defaults_clicked(b):
    
    
    sldr_rad.value=5
    sldr_len.value = 100
    sldr_nai.value= 14
    sldr_ki.value = 122.9
    sldr_cli.value = 5.2
    sldr_xi.value = 154.9
    sldr_zi.value = -0.85
    sldr_p.value = 0.1
    
btn_defaults.on_click(btn_defaults_clicked)


vbox_grand = widgets.VBox(children = [rad_box,len_box, hbox_nai,hbox_ki,hbox_cli,hbox_xi,hbox_zi,hbox_p,hbox_btns], layout=Layout(border ="solid"))

display(output1,vbox_grand)







Output()

# Set timing


In [32]:
global total_t,dt,t_arr

### TOTAL TIME
lbl_total_t = widgets.Label(value="Set the simulation run time (minutes):")

sldr_total_t = widgets.FloatSlider(value=1, min=0.5, max=20, step=0.5, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.2f')
HBox_total_t = widgets.HBox(children = [lbl_total_t,sldr_total_t])


### TIME STEP
lbl_dt = widgets.Label(value="Set the simulation time step (milliseconds):")
sldr_dt = widgets.FloatSlider(value=1, min=1, max=5, step=0.2, disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.2f')
HBox_dt = widgets.HBox(children = [lbl_dt,sldr_dt])

    
t_arr = [0]


vbox_t = widgets.VBox([HBox_total_t,HBox_dt], layout=Layout(border='solid'))



display(vbox_t)


#print("Set total simulation time (minutes):")

#print("Set time step (milliseconds):")

# Run Simulation

In [33]:

btn_sim = widgets.Button(description ="Run Simulation", button_style="Danger",disabled=False)
output_sim = widgets.Output()
output_prg = widgets.Output()


global SIM_OVER,na_arr, k_arr, cl_arr, x_arr, vm_arr, ecl_arr, ek_arr, w_arr, dna_arr, dk_arr, dcl_arr 
SIM_OVER = False

nao = 145e-3
clo = 119e-3
ko = 3.5e-3
xo = 29.5e-3


na_arr =[nai]
k_arr =[ki]
cl_arr =[cli]
x_arr =[xi]
vm_arr= [finvcar * (nai +ki + (zi*xi)-cli)]
ecl_arr =[RTF * np.log(ki/ko)]
ek_arr =[RTF *np.log(cli/clo)]
dna_arr =[0]
dk_arr =[0]
dcl_arr =[0]
w_arr =[w]





def btn_sim_clicked(b):
  
       
    global dt,total_t
    dt=sldr_dt.value*1e-3
    total_t= sldr_total_t.value *60
    interval = total_t/dt
    run_t=0
    finvcar = F / (cm*ar)
    
  
    prg = widgets.FloatProgress(description='Simulating...', min=0,step=dt,max=total_t,value=0,continuous_update=True)
    with output_prg:
        display(prg)
    
        

    while run_t < total_t:

        dna = 0
        dk = 0
        dcl = 0
        
        vm =  finvcar * (nai +ki + (zi*xi)-cli)
        ek =  RTF * np.log(ki/ko)
        ecl = RTF *np.log(cli/clo)
        
        jp = p * (nai/nao)**3
        jkkc2 = pkcc2 *(ek-ecl)
        
        dna = -dt * ar * (gna * (vm + RTF*np.log(nai/nao)) +3*jp ) 
        dk = -dt * ar * (gk * (vm + RTF*np.log(ki/ko)) -2*jp -jkk2) 
        dcl = dt * ar * (gcl * (vm + RTF*np.log(cli/clo)) +jkkc2 ) 
        
        na += dna
        k += dk
        cl += dcl
        
        
        osmi = nai + ki + cli +xi 
        osmo = nao + ko + clo +xo
        dw = dt * 0.018 * 0.018 *sa * (osmi - osmo)
        w2 = w + dw
        
        nai = nai * (w/w2)
        ki = ki * (w/w2)
        cli = cli * (w/w2)
        xi = xi * (w/w2)
        
        w =w2
        radius = np.sqrt(w/ (np.pi * length))
        sa = 2 * np.pi * radius * length
        ar = sa/w
        finvcar = F / (cm*ar)
        
        vm_arr.append(vm)
        ek_arr.append(ek)
        ecl_arr.append(ecl)
        na_arr.append(nai)
        k_arr.append(ki)
        cl_arr.append(cli)
        x_arr.append(xi)
        t_att.append(run_t)
               
        
        run_t += dt
        
        

       
    SIM_OVER = True
    btn_sim.disabled=True
    
    with output7:  
        print("Simulation complete!")
        
        
        



display(btn_sim,output_prg,output_sim)
        
btn_sim.on_click(btn_sim_clicked)  



Button(button_style='danger', description='Run Simulation', style=ButtonStyle())

Output()

Output()

UnboundLocalError: local variable 'ar' referenced before assignment